In [47]:
# million song dataset manipulation
import h5py
import numpy as np
import h5json
import sqlite3
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime

path = '/Users/andrewsang/datasci/thesis'

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

%matplotlib inline

In [48]:
import os

def walk_through_files(path, file_extension='.h5'):
    for (dirpath, dirnames, filenames) in os.walk(path):
        for filename in filenames:
            if filename.endswith(file_extension): 
                yield os.path.join(dirpath, filename)

In [49]:
# alternative method
def transform_files_to_arr(fname, p):
    # pull data
    fl = h5py.File(fname, 'r')
    segs_pitch = np.asarray(fl['analysis']['segments_pitches'])
    segs_start = np.asarray(fl['analysis']['segments_start'])
    segs_timbr = np.asarray(fl['analysis']['segments_timbre'])
    bpm = (len(np.asarray(fl['analysis']['beats_start'])) /
        np.asarray(fl['analysis']['segments_start'])[-1] / 60)
    
    # transform to pandas data frame
    df = pd.DataFrame(list(zip(segs_start, segs_pitch, segs_timbr)), columns=['time',1,2])
    for idx, m in zip([1,2], ['pitch_','timbr_']):
        ndf = pd.DataFrame(df[idx].to_list(), columns=[m+str(i+1) for i in np.arange(0,12)])
        df = pd.concat([df, ndf], axis=1)
    df = df.drop([1,2], axis=1)
    df.loc[:,'bpm'] = bpm
    
    origin = pd.to_datetime('2020-01-01') # set some arbitrary point so we can use pd.resample
    df.loc[:,'time'] = df['time'].map(lambda s: origin + pd.Timedelta(seconds=s))
    
    # 1/.10*60 = 600
    t_arr = (df.set_index('time')
               .resample('10ms').mean()
               .ffill()
               .resample('100ms').mean()
               .iloc[:p, :]
               .T.to_numpy())
    
    # shell (just in case something is less than 60 seconds long)
    fnl = np.zeros([25, p])
    fnl[:, :t_arr.shape[1]] = t_arr
    
    return fnl.reshape(1,25,-1)

In [50]:
# transform_files_to_arr('/storage/thesis/music/data/C/J/Y/TRCJYBG128F426613E.h5')

### Do it the .npy file way

In [51]:
# create a dictionary with track_id to genre data
genre_df = pd.read_csv('/storage/thesis/genres/style.csv')
genre_df.columns = ['track_id','genre']

genre_map = genre_df.set_index('track_id').to_dict()['genre']

In [52]:
# get track_id list of relevant ones
dt = pd.read_csv('test_train_split.csv')
test_train_vec = dt['track_id'].tolist()

In [53]:
pts = int(1/.1*60) # how many per second * number of seconds

In [54]:
# Save a small empty array
X_temp_fn = '/storage/thesis/music_ftr/temp_X.npy'
np.save(X_temp_fn, np.empty(1))

X_fn = '/storage/thesis/music_ftr/X_on_disk_v2.npy'
X_fn = '/storage/thesis/music_ftr/X_on_disk_samples_v2.npy'
X = np.memmap(X_temp_fn, dtype='float32', shape=(len(test_train_vec), 25, pts))

In [55]:
start = 0
path = '/storage/thesis/music/data/'
dirpaths = sorted([dirpath for (dirpath, dirnms, fnms) in os.walk(path, topdown=True)])
track_ids, genres = [], []

for dirpath in dirpaths:
    timestart = datetime.datetime.now()
    musics = np.empty((0, 25, pts), int)
    
    fs = [os.path.join(dirpath, f) for f in os.listdir(dirpath) 
          if os.path.isfile(os.path.join(dirpath, f)) and
              f[:-3] in test_train_vec]

    if len(fs) >= 1:
        for f in fs:
            tid = f.rsplit('/', 1)[-1].split('.h5')[0]
            if tid in test_train_vec:
                try:
                    gen = genre_map[tid]
                    music_data = transform_files_to_arr(f, pts)
                    # add to arrays
                    musics = np.nan_to_num(np.concatenate([musics, music_data]))
                    track_ids.append(tid)
                    genres.append(gen)
                except:
                    pass
            else:
                pass
        musics = musics.astype(np.float32)
        end = start + musics.shape[0]
        
        # fill it in
        X[start:end] = musics
        start = end
    timeend = datetime.datetime.now()
    print(dirpath, (timeend-timestart).microseconds / 1e6)

/storage/thesis/music/data/ 0.000584
/storage/thesis/music/data/A 0.000633
/storage/thesis/music/data/A/A 0.000533
/storage/thesis/music/data/A/A/A 0.486913
/storage/thesis/music/data/A/A/B 0.586197
/storage/thesis/music/data/A/A/C 0.190616
/storage/thesis/music/data/A/A/D 0.337555
/storage/thesis/music/data/A/A/E 0.175882
/storage/thesis/music/data/A/A/F 0.28322
/storage/thesis/music/data/A/A/G 0.488958
/storage/thesis/music/data/A/A/H 0.751163
/storage/thesis/music/data/A/A/I 0.362948
/storage/thesis/music/data/A/A/J 0.392443
/storage/thesis/music/data/A/A/K 0.246585
/storage/thesis/music/data/A/A/L 0.386161
/storage/thesis/music/data/A/A/M 0.484716
/storage/thesis/music/data/A/A/N 0.316375
/storage/thesis/music/data/A/A/O 0.302724
/storage/thesis/music/data/A/A/P 0.549757
/storage/thesis/music/data/A/A/Q 0.415046
/storage/thesis/music/data/A/A/R 0.337036
/storage/thesis/music/data/A/A/S 0.380968
/storage/thesis/music/data/A/A/T 0.489085
/storage/thesis/music/data/A/A/U 0.296472
/sto

/storage/thesis/music/data/A/H/F 0.74813
/storage/thesis/music/data/A/H/G 0.248774
/storage/thesis/music/data/A/H/H 0.238953
/storage/thesis/music/data/A/H/I 0.276327
/storage/thesis/music/data/A/H/J 0.314651
/storage/thesis/music/data/A/H/K 0.23294
/storage/thesis/music/data/A/H/L 0.373211
/storage/thesis/music/data/A/H/M 0.365052
/storage/thesis/music/data/A/H/N 0.198829
/storage/thesis/music/data/A/H/O 0.092214
/storage/thesis/music/data/A/H/P 0.239915
/storage/thesis/music/data/A/H/Q 0.250336
/storage/thesis/music/data/A/H/R 0.244817
/storage/thesis/music/data/A/H/S 0.303729
/storage/thesis/music/data/A/H/T 0.238584
/storage/thesis/music/data/A/H/U 0.590615
/storage/thesis/music/data/A/H/V 0.354917
/storage/thesis/music/data/A/H/W 0.317133
/storage/thesis/music/data/A/H/X 0.323536
/storage/thesis/music/data/A/H/Y 0.173501
/storage/thesis/music/data/A/H/Z 0.287005
/storage/thesis/music/data/A/I 0.008838
/storage/thesis/music/data/A/I/A 0.353297
/storage/thesis/music/data/A/I/B 0.344

/storage/thesis/music/data/A/O/M 0.468616
/storage/thesis/music/data/A/O/N 0.522531
/storage/thesis/music/data/A/O/O 0.273643
/storage/thesis/music/data/A/O/P 0.15117
/storage/thesis/music/data/A/O/Q 0.229566
/storage/thesis/music/data/A/O/R 0.462597
/storage/thesis/music/data/A/O/S 0.665611
/storage/thesis/music/data/A/O/T 0.304529
/storage/thesis/music/data/A/O/U 0.332133
/storage/thesis/music/data/A/O/V 0.232241
/storage/thesis/music/data/A/O/W 0.298749
/storage/thesis/music/data/A/O/X 0.375764
/storage/thesis/music/data/A/O/Y 0.34126
/storage/thesis/music/data/A/O/Z 0.240342
/storage/thesis/music/data/A/P 0.009485
/storage/thesis/music/data/A/P/A 0.252379
/storage/thesis/music/data/A/P/B 0.241768
/storage/thesis/music/data/A/P/C 0.429705
/storage/thesis/music/data/A/P/D 0.34079
/storage/thesis/music/data/A/P/E 0.498905
/storage/thesis/music/data/A/P/F 0.391767
/storage/thesis/music/data/A/P/G 0.410302
/storage/thesis/music/data/A/P/H 0.322921
/storage/thesis/music/data/A/P/I 0.3516

/storage/thesis/music/data/A/V/U 0.210696
/storage/thesis/music/data/A/V/V 0.208285
/storage/thesis/music/data/A/V/W 0.203432
/storage/thesis/music/data/A/V/X 0.368024
/storage/thesis/music/data/A/V/Y 0.219113
/storage/thesis/music/data/A/V/Z 0.393026
/storage/thesis/music/data/A/W 0.269994
/storage/thesis/music/data/A/W/A 0.178516
/storage/thesis/music/data/A/W/B 0.563554
/storage/thesis/music/data/A/W/C 0.5359
/storage/thesis/music/data/A/W/D 0.287201
/storage/thesis/music/data/A/W/E 0.354239
/storage/thesis/music/data/A/W/F 0.272822
/storage/thesis/music/data/A/W/G 0.323214
/storage/thesis/music/data/A/W/H 0.25207
/storage/thesis/music/data/A/W/I 0.322172
/storage/thesis/music/data/A/W/J 0.223068
/storage/thesis/music/data/A/W/K 0.222962
/storage/thesis/music/data/A/W/L 0.183448
/storage/thesis/music/data/A/W/M 0.345145
/storage/thesis/music/data/A/W/N 0.415239
/storage/thesis/music/data/A/W/O 0.339217
/storage/thesis/music/data/A/W/P 0.312298
/storage/thesis/music/data/A/W/Q 0.2646

/storage/thesis/music/data/B/D/A 0.313815
/storage/thesis/music/data/B/D/B 0.427665
/storage/thesis/music/data/B/D/C 0.319089
/storage/thesis/music/data/B/D/D 0.196437
/storage/thesis/music/data/B/D/E 0.224592
/storage/thesis/music/data/B/D/F 0.141075
/storage/thesis/music/data/B/D/G 0.368416
/storage/thesis/music/data/B/D/H 0.562617
/storage/thesis/music/data/B/D/I 0.540535
/storage/thesis/music/data/B/D/J 0.348123
/storage/thesis/music/data/B/D/K 0.187925
/storage/thesis/music/data/B/D/L 0.438753
/storage/thesis/music/data/B/D/M 0.561469
/storage/thesis/music/data/B/D/N 0.448934
/storage/thesis/music/data/B/D/O 0.610837
/storage/thesis/music/data/B/D/P 0.535053
/storage/thesis/music/data/B/D/Q 0.38738
/storage/thesis/music/data/B/D/R 0.289851
/storage/thesis/music/data/B/D/S 0.628495
/storage/thesis/music/data/B/D/T 0.645645
/storage/thesis/music/data/B/D/U 0.25025
/storage/thesis/music/data/B/D/V 0.555971
/storage/thesis/music/data/B/D/W 0.149041
/storage/thesis/music/data/B/D/X 0.2

...

/storage/thesis/music/data/Y/W/N 0.403957
/storage/thesis/music/data/Y/W/O 0.231936
/storage/thesis/music/data/Y/W/P 0.585355
/storage/thesis/music/data/Y/W/Q 0.374444
/storage/thesis/music/data/Y/W/R 0.447241
/storage/thesis/music/data/Y/W/S 0.260303
/storage/thesis/music/data/Y/W/T 0.365686
/storage/thesis/music/data/Y/W/U 0.632495
/storage/thesis/music/data/Y/W/V 0.276185
/storage/thesis/music/data/Y/W/W 0.212141
/storage/thesis/music/data/Y/W/X 0.291211
/storage/thesis/music/data/Y/W/Y 0.377176
/storage/thesis/music/data/Y/W/Z 0.421728
/storage/thesis/music/data/Y/X 0.001756
/storage/thesis/music/data/Y/X/A 0.498856
/storage/thesis/music/data/Y/X/B 0.155348
/storage/thesis/music/data/Y/X/C 0.46823
/storage/thesis/music/data/Y/X/D 0.430493
/storage/thesis/music/data/Y/X/E 0.2751
/storage/thesis/music/data/Y/X/F 0.389876
/storage/thesis/music/data/Y/X/G 0.197801
/storage/thesis/music/data/Y/X/H 0.487507
/storage/thesis/music/data/Y/X/I 0.578056
/storage/thesis/music/data/Y/X/J 0.4852

/storage/thesis/music/data/Z/D/U 0.37724
/storage/thesis/music/data/Z/D/V 0.316449
/storage/thesis/music/data/Z/D/W 0.470749
/storage/thesis/music/data/Z/D/X 0.54507
/storage/thesis/music/data/Z/D/Y 0.450502
/storage/thesis/music/data/Z/D/Z 0.219777
/storage/thesis/music/data/Z/E 0.001915
/storage/thesis/music/data/Z/E/A 0.118612
/storage/thesis/music/data/Z/E/B 0.275985
/storage/thesis/music/data/Z/E/C 0.517483
/storage/thesis/music/data/Z/E/D 0.357282
/storage/thesis/music/data/Z/E/E 0.523799
/storage/thesis/music/data/Z/E/F 0.263708
/storage/thesis/music/data/Z/E/G 0.322387
/storage/thesis/music/data/Z/E/H 0.308822
/storage/thesis/music/data/Z/E/I 0.593049
/storage/thesis/music/data/Z/E/J 0.523671
/storage/thesis/music/data/Z/E/K 0.276046
/storage/thesis/music/data/Z/E/L 0.203943
/storage/thesis/music/data/Z/E/M 0.410333
/storage/thesis/music/data/Z/E/N 0.407359
/storage/thesis/music/data/Z/E/O 0.566383
/storage/thesis/music/data/Z/E/P 0.390444
/storage/thesis/music/data/Z/E/Q 0.927

/storage/thesis/music/data/Z/L/A 0.456382
/storage/thesis/music/data/Z/L/B 0.209797
/storage/thesis/music/data/Z/L/C 0.455878
/storage/thesis/music/data/Z/L/D 0.146844
/storage/thesis/music/data/Z/L/E 0.183053
/storage/thesis/music/data/Z/L/F 0.209059
/storage/thesis/music/data/Z/L/G 0.412162
/storage/thesis/music/data/Z/L/H 0.326203
/storage/thesis/music/data/Z/L/I 0.348121
/storage/thesis/music/data/Z/L/J 0.232413
/storage/thesis/music/data/Z/L/K 0.362503
/storage/thesis/music/data/Z/L/L 0.273757
/storage/thesis/music/data/Z/L/M 0.185339
/storage/thesis/music/data/Z/L/N 0.187816
/storage/thesis/music/data/Z/L/O 0.526859
/storage/thesis/music/data/Z/L/P 0.479807
/storage/thesis/music/data/Z/L/Q 0.363435
/storage/thesis/music/data/Z/L/R 0.195828
/storage/thesis/music/data/Z/L/S 0.319314
/storage/thesis/music/data/Z/L/T 0.629121
/storage/thesis/music/data/Z/L/U 0.311391
/storage/thesis/music/data/Z/L/V 0.429046
/storage/thesis/music/data/Z/L/W 0.193389
/storage/thesis/music/data/Z/L/X 0

/storage/thesis/music/data/Z/S/H 0.166038
/storage/thesis/music/data/Z/S/I 0.758087
/storage/thesis/music/data/Z/S/J 0.318429
/storage/thesis/music/data/Z/S/K 0.603694
/storage/thesis/music/data/Z/S/L 0.37596
/storage/thesis/music/data/Z/S/M 0.240971
/storage/thesis/music/data/Z/S/N 0.566441
/storage/thesis/music/data/Z/S/O 0.491423
/storage/thesis/music/data/Z/S/P 0.329877
/storage/thesis/music/data/Z/S/Q 0.070061
/storage/thesis/music/data/Z/S/R 0.280848
/storage/thesis/music/data/Z/S/S 0.409569
/storage/thesis/music/data/Z/S/T 0.785758
/storage/thesis/music/data/Z/S/U 0.196584
/storage/thesis/music/data/Z/S/V 0.534117
/storage/thesis/music/data/Z/S/W 0.345142
/storage/thesis/music/data/Z/S/X 0.315526
/storage/thesis/music/data/Z/S/Y 0.188394
/storage/thesis/music/data/Z/S/Z 0.42819
/storage/thesis/music/data/Z/T 0.002198
/storage/thesis/music/data/Z/T/A 0.289738
/storage/thesis/music/data/Z/T/B 0.352083
/storage/thesis/music/data/Z/T/C 0.250276
/storage/thesis/music/data/Z/T/D 0.273

/storage/thesis/music/data/Z/Z/O 0.474541
/storage/thesis/music/data/Z/Z/P 0.252669
/storage/thesis/music/data/Z/Z/Q 0.301288
/storage/thesis/music/data/Z/Z/R 0.198284
/storage/thesis/music/data/Z/Z/S 0.35262
/storage/thesis/music/data/Z/Z/T 0.26285
/storage/thesis/music/data/Z/Z/U 0.29356
/storage/thesis/music/data/Z/Z/V 0.062731
/storage/thesis/music/data/Z/Z/W 0.310068
/storage/thesis/music/data/Z/Z/X 0.23236
/storage/thesis/music/data/Z/Z/Y 0.373257
/storage/thesis/music/data/Z/Z/Z 0.590977


In [56]:
# save everything to disk
np.save(X_fn, X)
np.save('/storage/thesis/music_ftr/genres_samples_v2.npy', np.asarray(genres, dtype=object))
np.save('/storage/thesis/music_ftr/track_ids_samples_v2.npy', np.asarray(track_ids, dtype=object))

In [57]:
# 7:45pm
os.remove(X_temp_fn)

In [46]:
ls -lh /storage/thesis/music_ftr/X_on_disk_samples.npy

-rw-r--r-- 1 root root 4.0G Sep  5 15:38 /storage/thesis/music_ftr/X_on_disk_samples.npy
